In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###초기설정

In [15]:
!pip install fastapi uvicorn sentence-transformers pdfplumber openai faiss-gpu

In [42]:
!pip install nest_asyncio uvicorn

In [35]:
pip install python-multipart

In [169]:
import os
import pdfplumber
import faiss
import numpy as np
from fastapi import FastAPI, UploadFile, File, Form, HTTPException
from pydantic import BaseModel
from typing import List
from sentence_transformers import SentenceTransformer
from openai import OpenAI
from fastapi.middleware.cors import CORSMiddleware
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import base64
import json

In [180]:
ngrok.set_auth_token("")

In [181]:
app = FastAPI()
sbert_model = SentenceTransformer('all-MPNet-Base-V2')
os.environ["OPENAI_API_KEY"] = ""
client = OpenAI()


In [182]:
# CORS 설정
app.add_middleware(
    CORSMiddleware,
    allow_origins="http://127.0.0.1:5500",  # 허용할 출처 설정
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [183]:
# 4. 벡터 DB(FAISS) 초기화 (GPU 기반)
dimension = 768
res = faiss.StandardGpuResources()  # GPU 리소스 생성
index = faiss.GpuIndexFlatL2(res, dimension)  # L2 거리 기반 GPU 인덱스

text_chunks = []
chunk_metadata = []

pdf_folder_path = '/content/drive/MyDrive/Colab Notebooks/dataset'
pdf_files = [f for f in os.listdir(pdf_folder_path) if f.endswith('.pdf')]

In [184]:
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
    return text

def split_text_into_chunks(text, max_tokens=256):
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_tokens):
        chunk = ' '.join(words[i:i + max_tokens])
        chunks.append(chunk)
    return chunks

def get_embeddings(texts):
    try:
        # texts가 비어있는 경우 처리
        if not texts:
            return np.array([], dtype=np.float32)

        # embeddings 생성
        embeddings = sbert_model.encode(texts)

        # numpy array로 변환하고 float32로 타입 변환
        embeddings_array = np.array(embeddings, dtype=np.float32)

        print(f"임베딩 생성 완료. shape: {embeddings_array.shape}, dtype: {embeddings_array.dtype}")
        return embeddings_array
    except Exception as e:
        print(f"임베딩 생성 중 오류 발생: {str(e)}")
        raise

In [185]:

for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_folder_path, pdf_file)
    print(f"Processing {pdf_file}...")

    pdf_text = extract_text_from_pdf(pdf_path)
    chunks = split_text_into_chunks(pdf_text)

    if len(chunks) == 0:
        print(f"PDF 파일 {pdf_file}에서 텍스트를 추출하지 못했습니다. 건너뜁니다.")
        continue

    embeddings = get_embeddings(chunks)
    embeddings = np.array(embeddings)  # FAISS 호환을 위해 Numpy 배열로 변환

    # FAISS 인덱스 추가 전 데이터 차원 확인
    print(f"FAISS에 추가할 임베딩 데이터 크기: {embeddings.shape}")
    assert embeddings.shape[1] == dimension, f"임베딩 차원 {embeddings.shape[1]}이 FAISS 설정 차원 {dimension}과 일치하지 않습니다."

    index.add(embeddings)  # 벡터 DB에 추가

    text_chunks.extend(chunks)
    chunk_metadata.extend([{"file": pdf_file, "chunk_index": i} for i in range(len(chunks))])

print(f"총 처리된 청크 개수: {len(text_chunks)}")
print(f"FAISS 인덱스에 저장된 벡터 개수: {index.ntotal}")

Processing data1.pdf...
임베딩 생성 완료. shape: (5, 768), dtype: float32
FAISS에 추가할 임베딩 데이터 크기: (5, 768)
Processing data2.pdf...
임베딩 생성 완료. shape: (5, 768), dtype: float32
FAISS에 추가할 임베딩 데이터 크기: (5, 768)
총 처리된 청크 개수: 10
FAISS 인덱스에 저장된 벡터 개수: 10


In [113]:
!pip install python-multipart

In [186]:
class PDFUploadRequest(BaseModel):
    filename: str
    base64_data: str

@app.post("/upload")
async def upload_pdf(request: PDFUploadRequest):
    try:
        # base64 데이터를 바이너리로 디코딩
        file_data = base64.b64decode(request.base64_data.split(',')[1] if ',' in request.base64_data else request.base64_data)

        # 파일 저장
        file_path = f"./uploads/{request.filename}"
        os.makedirs("./uploads", exist_ok=True)

        with open(file_path, "wb") as f:
            f.write(file_data)

        try:
            # PDF 텍스트 추출 및 처리
            pdf_text = extract_text_from_pdf(file_path)
            if not pdf_text.strip():  # 빈 텍스트 체크
                raise HTTPException(status_code=400, detail="No text could be extracted from the PDF")

            chunks = split_text_into_chunks(pdf_text)
            if not chunks:  # 빈 청크 체크
                raise HTTPException(status_code=400, detail="No text chunks were generated")

            embeddings = get_embeddings(chunks)

            # embeddings가 비어있지 않은 경우에만 처리
            if embeddings.size > 0:
                # FAISS index에 추가
                try:
                    index.add(embeddings)
                    print(f"FAISS index에 {len(embeddings)} 개의 벡터 추가됨")
                except Exception as e:
                    print(f"FAISS index 추가 중 오류: {str(e)}")
                    raise HTTPException(status_code=500, detail=f"Error adding to FAISS index: {str(e)}")

                # 메타데이터 추가
                text_chunks.extend(chunks)
                chunk_metadata.extend([{
                    "file": request.filename,
                    "chunk_index": i
                } for i in range(len(chunks))])

            return {"message": "200", "filename": request.filename}

        finally:
            # 임시 파일 삭제
            if os.path.exists(file_path):
                os.remove(file_path)

    except Exception as e:
        print(f"Error: {str(e)}")
        raise HTTPException(status_code=400, detail=str(e))

In [187]:
class AskRequest(BaseModel):
    query: str
    top_k: int = 5  # 기본값 설정

@app.post("/ask/")
async def ask_question(request: AskRequest):
    query = request.query
    top_k = request.top_k

    # 1. 사용자 질문을 SBERT 임베딩으로 변환
    query_embedding = get_embeddings([query])[0]

    # 2. 벡터 DB에서 관련 텍스트 검색
    distances, indices = index.search(np.array([query_embedding]), k=top_k)
    results = [{"text": text_chunks[i], "metadata": chunk_metadata[i], "distance": distances[0][j]}
               for j, i in enumerate(indices[0])]

    # 3. 검색된 텍스트를 문맥으로 연결
    context = "\n".join([result["text"] for result in results])

    # 4. OpenAI GPT를 사용해 응답 생성
    prompt = f"문맥: {context}\n\n질문: {query}\n\n답변:"
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "당신은 문맥 기반 정보를 활용해 사용자 질문에 정확하고 친절하게 답변하는 한국어 전문가입니다."
            },
            {
                "role": "user",
                "content": prompt
            }
        ]
    )
    answer = completion.choices[0].message

    # 5. 응답 반환
    return {"answer": answer}

In [188]:
nest_asyncio.apply()

In [ ]:
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# FastAPI 서버 실행
uvicorn.run(app, host="0.0.0.0", port=8000)

Public URL: NgrokTunnel: "https://1c2c-34-16-217-63.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [208]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     210.95.155.217:0 - "OPTIONS /upload HTTP/1.1" 200 OK
임베딩 생성 완료. shape: (5, 768), dtype: float32
FAISS index에 5 개의 벡터 추가됨
INFO:     210.95.155.217:0 - "POST /upload HTTP/1.1" 200 OK
INFO:     210.95.155.217:0 - "OPTIONS /ask/ HTTP/1.1" 200 OK
임베딩 생성 완료. shape: (1, 768), dtype: float32
INFO:     210.95.155.217:0 - "POST /ask/ HTTP/1.1" 200 OK
